# RAG For Invoices

## Set Up

In [1]:
!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured
!brew install poppler-utils
!pip install pdfminer.six
!brew install tesseract-ocr
!brew install libtesseract-dev
!pip install pytesseract
!pip install pillow_heif
!pip3 install opencv-python
!pip install pikepdf
!pip install unstructured_pytesseract
!pip install unstructured_inference
!pip install unstructured_inference
!pip install sentence-transformers

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
==> Searching for similarly named formulae and casks...
==> Formulae
poppler-qt5

To install poppler-qt5, run:
  brew install poppler-qt5
==> Searching for similarly named formulae and casks...
==> Formulae
tesseract ✔                              tesseract-lang

To install tesseract ✔, run:
  brew install tesseract ✔
==> Searching for similarly named formulae and casks...
==> Formulae
tesseract ✔

To install tesseract ✔, run:
  brew install tesseract ✔
  Using cached torch-2.3.0-cp311-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl (59.7 MB)
  Using cached torch-2.3.0-cp311-none

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.retrievers import BM25Retriever, EnsembleRetriever
import os
from getpass import getpass
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from google.cloud import storage #!pip install google-cloud-storage
import tempfile

In [3]:
# Hugging Face token is imported from another file
# to ensure safety
from alessandra_secrets import hugging_face_token

os.environ["HUGGINGFACEHUB_API_TOKEN"] = hugging_face_token

hf_token = hugging_face_token

## Connection to GCP (Downloading data from bucket)

With the following code we download the file from the bucket, we create the embeddings and save them in the croma vector store

In [4]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

# Path to your service account key file
key_path = '/Users/alessandracerutti/ucl-invoice-assistant/ucl-engineering-invoice-5d941af64425.json'
credentials = Credentials.from_service_account_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/cloud-platform'])

if credentials.expired:
    credentials.refresh(Request())
    

In [8]:
def download_files(bucket_name, hf_token):
    # Initialize Google Cloud Storage client
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)

    # List for document contents
    docs = []

    # Download each file from the bucket
    blobs = list(bucket.list_blobs())
    for blob in blobs:
        # Create a temporary file
        _, temp_local_filename = tempfile.mkstemp()

        # Download the file from the bucket
        blob.download_to_filename(temp_local_filename)

        # Load content using UnstructuredPDFLoader for each file
        pdf_loader = UnstructuredPDFLoader(temp_local_filename)
        loaded_data = pdf_loader.load()

        # Remove the temporary file
        os.remove(temp_local_filename)
        # Append a new dictionary for the entire document content
        docs.append({
            'page_content': loaded_data[0].page_content,  
            'metadata': {
                'source': blob.name
            }
        })
    return docs

In [34]:
# Instantiates a client
bucket_name = "invoice_scan"
vectorstore = download_files(bucket_name, hf_token)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [36]:
bucket_name = "bank_statement_scan"
vectorstore1 = download_files(bucket_name, hf_token)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [37]:
# getting bank and invoice together in a variable: 
vectorstore = vectorstore+vectorstore1

In [38]:
vectorstore

[{'page_content': "Restaurant du CLUB NAUTIQUE MORGIFN\n\nPI. de la Navigation 1\n\n1110 Morges Facture 983 167 Numéro de client 2'262 F3A Collaborateur Rosanna —SC“C:sSCSC‘SS Novazzano 21.02.2024 Page 1/2 Adresse de fourniture: Restaurant du, CLUB NAUTIQUE MORGIEN, B. de la Navigation 1, 1110 Morges\n\nFD (P)-posta1\n\nNo. article / désignation Quantité E201G Café Extra-Milano en grains 24.00 pcs. 23.00 552.00 23 1000g No lot 24LT16FE1, 24.00 pcs. 16.08.2025 Total intermédiaire 552.00 TVA 2.6% net (Code 23) de 552.00 14.35\n\nTotal CHF\n\nConditions de paiement: 30 jours net.",
  'metadata': {'source': 'invoice_1.pdf'}},
 {'page_content': "Verein Berufliche Integration CHANCE Z! Niklaus-Thus-Platz 19\n\n4800 Zofingen Rechnung 983 986 Kundennummer 11'584 Dgs Sachebearbeiter Rosanna Novazzano 11.03.2024 Seite 1/2 Liefersadresse: Verein, Berufliche Integration CHANCE Z!, Niklaus-Thus-Platz 19, 4800 Zofingen FD - Posta 1 (Zahlung: Bankueberweisung)\n\nSPEDIRE A: Wunschladen.ch, Herr Peter

### Recursive Splitter
We select a small chunck size because we want to catch the informations in the invoices

In [39]:
# now let's create a function to split the texts in chuncks 
def text_splitter_fun(vectorstore): 
    texts = [doc['page_content'] for doc in vectorstore] # get the texts
    
    # initialise the text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, 
                                               chunk_overlap=50, 
                                               separators=["\n\n", "\n", "(?<=\. )", " ", ""])
    
    # create documents from the texts 
    creating_doc = text_splitter.create_documents(texts) 
    
    # apply the splitter
    chunks = text_splitter.split_documents(creating_doc)

    return chunks 

In [40]:
chunks = text_splitter_fun(vectorstore)

Let's look into the chuncks, we should see similar texts as before but more split up.

In [45]:
chunks

[Document(page_content='Restaurant du CLUB NAUTIQUE MORGIFN\n\nPI. de la Navigation 1'),
 Document(page_content="1110 Morges Facture 983 167 Numéro de client 2'262 F3A Collaborateur Rosanna —SC“C:sSCSC‘SS Novazzano 21.02.2024 Page 1/2 Adresse de fourniture:"),
 Document(page_content='21.02.2024 Page 1/2 Adresse de fourniture: Restaurant du, CLUB NAUTIQUE MORGIEN, B. de la Navigation 1, 1110 Morges'),
 Document(page_content='FD (P)-posta1'),
 Document(page_content='No. article / désignation Quantité E201G Café Extra-Milano en grains 24.00 pcs. 23.00 552.00 23 1000g No lot 24LT16FE1, 24.00 pcs. 16.08.2025 Total'),
 Document(page_content='No lot 24LT16FE1, 24.00 pcs. 16.08.2025 Total intermédiaire 552.00 TVA 2.6% net (Code 23) de 552.00 14.35'),
 Document(page_content='Total CHF\n\nConditions de paiement: 30 jours net.'),
 Document(page_content='Verein Berufliche Integration CHANCE Z! Niklaus-Thus-Platz 19'),
 Document(page_content="4800 Zofingen Rechnung 983 986 Kundennummer 11'584 Dgs S

### Embeddings & Saving in Vectore Store Chroma
Now we want to create the embeddings and save them in the vector store

In [41]:
# create embeddings and store them in the chroma db
def create_embeddings_and_save(chunks, model_name="all-MiniLM-L6-v2", persist_directory="./chroma_db"):
    # Select Embedding Model
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create embeddings and save them in chroma
    vectorstore = Chroma.from_documents(chunks, embedding_function, persist_directory="./chroma_db")

    return vectorstore

In [42]:
# investigating local variables for the chroma database 
print(os.getenv('KMP_DEVICE_THREAD_LIMIT'))
print(os.getenv('KMP_TEAMS_THREAD_LIMIT'))
print(os.getenv('OMP_THREAD_LIMIT'))

None
None
1


In [43]:
# setting the thread limit to avoid error messages
OMP_THREAD_LIMIT = 8

In [44]:
vectorstore = create_embeddings_and_save(chunks)

Now we want to try a similarity search, let's look for date and analyse how the three documents are similar to each other

In [55]:
query = "date"
docs = vectorstore.similarity_search(query)

# print results
print("============================================")
print("Doc number one: \n",docs[0].page_content)
print("============================================")
print("Doc number two: \n",docs[1].page_content)
print("============================================")
print("Doc number three: \n",docs[2].page_content)

Doc number one: 
 18.03.2024

18.03.2024

18.03.2024

18.03.2024

Visualizzato in UBS e-banking il 27.03.2024, 15:24:12 GMT
Doc number two: 
 CHF Conto corrente aziende UBS Registrazione

Date Descrizione

Data dell'operazione 11.03.2024 — Accredito riferimento QR Registrazione 11.03.2024
Doc number three: 
 Registrazione Valutazione in CHF Date Descrizione N. di transazione Importo contabile.


## Retriever
Initialise a retriever

In [82]:
# create a function for the ensemble retriever
def create_ensemble(vectorstore=vectorstore, chunks = chunks):
    retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 10}) # similarity retriever
    keyword_retriever = BM25Retriever.from_documents(chunks) # key words retriever
    keyword_retriever.k =  2 #
    ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])
    return ensemble_retriever



In [83]:
# call the function and create the retriver
ensemble_retriever = create_ensemble()

## Initialise LLM

In [84]:
from alessandra_secrets import hugging_face_token

os.environ["HUGGINGFACEHUB_API_TOKEN"]= hugging_face_token

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.5,"max_new_tokens":1024}
)

template = """
<|system|>>
You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

Keep in mind, you will lose the job, if you answer out of CONTEXT questions

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [85]:
# Environment SetUp Function
def set_environment(api_token):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_token


In [98]:
# Model Initialisation Function
from alessandra_secrets import hugging_face_token

def initialize_model(repo_id = "mistralai/Mistral-7B-Instruct-v0.1", temperature=0.6, max_new_tokens=1024):
    return HuggingFaceHub(
        repo_id=repo_id,
        model_kwargs={"temperature": temperature, "max_new_tokens": max_new_tokens}
    )


### Function to define the chain

In [99]:
def construct_chain(retriever = ensemble_retriever, prompt=prompt, llm=llm, output_parser=StrOutputParser()):
    chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
        )
    return chain
    

### Setting Everything Up together

In [100]:
# Set environment
set_environment(hugging_face_token)

# Initialize model
llm = initialize_model(repo_id="mistralai/Mistral-7B-Instruct-v0.1")

# Create prompt template
from prompt_template import prompt_template_ale # template is stored as ChatPromptTemplate in the template library

prompt = ChatPromptTemplate.from_template(prompt_template_ale)

# Create the chain
chain = construct_chain(retriever = ensemble_retriever, 
                        prompt=prompt, 
                        llm=llm, 
                        output_parser=output_parser)


### Interact with Mistral AI

In [101]:
print(chain.invoke("Can you please provide me with a summary of the products in the documents?"))

Human: 
    >
    You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
    You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
    Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

    Keep in mind, you will lose the job, if you answer out of CONTEXT questions

    CONTEXT: [Document(page_content='Menge : Einzelpreis Rabatt % Preis 15 Kaffee Einzelportionen Koffeinfrei 4.00 Box 7.80 5.00 29.65 23 Box 20Stk Chargen-Nr. M22A, 31.01.2026 4.00 Box'), Document(page_content='Quantita\n\n4.00 pz\n\n4.00 pz\n\n1.00 pz\n\n1.00 pz\n\n1.00 box\n\n1.00 box\n\n1.00 pz\n\n1.00 pz\n\ndi\n\nSnack Bar\n\nIL RIFUGIO Sagl Riva Caccia 1D\n\n6900 Lugano'), Document(page_content='NP Professional caffé "Espresso" 10.00 box box 50pz Lotto diproduzione 0, 08.08.2025 10.00 box NP Professional caffé "Fine Cup" 2.00

In [103]:
print(chain.invoke("At what price did we sell the Einzelportionen Koffeinfrei Box?"))

Human: 
    >
    You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
    You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
    Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

    Keep in mind, you will lose the job, if you answer out of CONTEXT questions

    CONTEXT: [Document(page_content='Menge : Einzelpreis Rabatt % Preis 15 Kaffee Einzelportionen Koffeinfrei 4.00 Box 7.80 5.00 29.65 23 Box 20Stk Chargen-Nr. M22A, 31.01.2026 4.00 Box'), Document(page_content='NP Professional caffé "Espresso" 10.00 box box 50pz Lotto diproduzione 0, 08.08.2025 10.00 box NP Professional caffé "Fine Cup" 2.00 box Omaggio'), Document(page_content='Quantita\n\n4.00 pz\n\n4.00 pz\n\n1.00 pz\n\n1.00 pz\n\n1.00 box\n\n1.00 box\n\n1.00 pz\n\n1.00 pz\n\ndi\n\nSnack Bar\n\nIL RIFUGIO Sagl Riva Caccia 1D\n\

In [92]:
print(chain.invoke("Did all the customers pay the invoices?"))

Human: 
    >
    You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
    You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
    Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

    Keep in mind, you will lose the job, if you answer out of CONTEXT questions

    CONTEXT: [Document(page_content='No lot 24LT16FE1, 24.00 pcs. 16.08.2025 Total intermédiaire 552.00 TVA 2.6% net (Code 23) de 552.00 14.35'), Document(page_content='Chargen-Nr. 23-03768, 3.00 Stk\n\n31.07.2024 Zwischentotal 200.00 MWST 2.6% Netto (Code 23) von 200.00 5.20 Total CHF 205.20'), Document(page_content='No. article / désignation Quantité E201G Café Extra-Milano en grains 24.00 pcs. 23.00 552.00 23 1000g No lot 24LT16FE1, 24.00 pcs. 16.08.2025 Total'), Document(page_content='Bonifico Multimat\n\nBELARMINO MARTINEZ\n\n

In [105]:
print(chain.invoke("Could you please identify the first invoice that was settled? Review the bank statements to determine the initial payment that correlates with any of the invoices."))

Human: 
    >
    You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
    You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
    Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

    Keep in mind, you will lose the job, if you answer out of CONTEXT questions

    CONTEXT: [Document(page_content="Fattura\n\n983 360\n\nNumero cliente 7'726\n\nCollaboratore\n\nIsabella\n\nIndirizzo di fornitura:\n\nFD - Pagamento E-Banking\n\nNumero articolo / Descrizione"), Document(page_content='Chargen-Nr. 23-03768, 3.00 Stk\n\n31.07.2024 Zwischentotal 200.00 MWST 2.6% Netto (Code 23) von 200.00 5.20 Total CHF 205.20'), Document(page_content='Chargen-Nr. 3518, 08.09.2025\n\n1.00 Krt. 6Stk.\n\nRechnung 981 341\n\nNovazzano 10.01.2024 Seite 3/4'), Document(page_content='332390000000009789840083 639\n\nVis

In [93]:
print(chain.invoke("Please check for invoices that have a high amount and then determine which one has the highest amount"))

Human: 
    >
    You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
    You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
    Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

    Keep in mind, you will lose the job, if you answer out of CONTEXT questions

    CONTEXT: [Document(page_content='No lot 24LT16FE1, 24.00 pcs. 16.08.2025 Total intermédiaire 552.00 TVA 2.6% net (Code 23) de 552.00 14.35'), Document(page_content='M101G Kaffee Milano Bohnen 8.00 Stk. 25.00 200.00 23 1000g Chargen-Nr. 24LT15FE1, 8.00 Stk. 15.08.2025'), Document(page_content='Chargen-Nr. 23-03768, 3.00 Stk\n\n31.07.2024 Zwischentotal 200.00 MWST 2.6% Netto (Code 23) von 200.00 5.20 Total CHF 205.20'), Document(page_content='Bonifico Multimat\n\nBELARMINO MARTINEZ\n\nCH 1800 VEVEY\n\n983703\n\nAwiso di accredito

In [94]:
print(chain.invoke("In what language are the invoices?"))

Human: 
    >
    You are an AI Assistant for the accounting department of a coffee company that follows instructions extremely well.
    You are in charge to support the invoice team of the coffee company. You look through all the invoices before answering the questions.
    Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT.

    Keep in mind, you will lose the job, if you answer out of CONTEXT questions

    CONTEXT: [Document(page_content='Quantita\n\n4.00 pz\n\n4.00 pz\n\n1.00 pz\n\n1.00 pz\n\n1.00 box\n\n1.00 box\n\n1.00 pz\n\n1.00 pz\n\ndi\n\nSnack Bar\n\nIL RIFUGIO Sagl Riva Caccia 1D\n\n6900 Lugano'), Document(page_content="Fattura\n\n983 360\n\nNumero cliente 7'726\n\nCollaboratore\n\nIsabella\n\nIndirizzo di fornitura:\n\nFD - Pagamento E-Banking\n\nNumero articolo / Descrizione"), Document(page_content='M101G Kaffee Milano Bohnen 8.00 Stk. 25.00 200.00 23 1000g Chargen-Nr. 24LT15FE1, 8.00 Stk. 15.08.2025'), Document(page_c